In [318]:
from datetime import datetime
import sqlite3
import pandas as pd
import plotly.express as px


In [319]:
# Helper functions
def get_icd9_description(icd9_code):
    return diagnosis_labes[diagnosis_labes['ICD9_CODE'] == icd9_code]['LONG_TITLE'].values[0]

def calculate_age(dob, addmission_date):
    dob_date = datetime.strptime(dob, '%Y-%m-%d %H:%M:%S')
    addmission_date = datetime.strptime(addmission_date, '%Y-%m-%d %H:%M:%S')  
    age = (addmission_date - dob_date).days // 365
    return age

In [320]:

# Connect to the SQLite database
conn = sqlite3.connect('../Assignment_MIMIC_SQL/database.db')

# Query to get ICU stays with patient information (including date of birth, expiry flag, and date of death)
query = """
SELECT icu.subject_id, icu.intime, icu.outtime, p.dob, p.expire_flag, p.dod
FROM icustays icu
JOIN patients p ON icu.subject_id = p.subject_id;
"""

# Load the query result into a pandas DataFrame
merged_df = pd.read_sql(query, conn)

# Close the connection after the query
conn.close()

# Convert 'intime' and 'outtime' to datetime
merged_df['INTIME'] = pd.to_datetime(merged_df['INTIME'])
merged_df['OUTTIME'] = pd.to_datetime(merged_df['OUTTIME'])

# Calculate ICU stay duration (LOS) in hours
merged_df['STAY_DURATION'] = (merged_df['OUTTIME'] - merged_df['INTIME']).dt.total_seconds() / (60 * 60)

# Calculate patient's age at ICU admission (in years)
merged_df['AGE_AT_ADDMISSION'] = merged_df.apply(lambda row: calculate_age(str(row['DOB']), str(row['INTIME'])), axis=1)

# Determine mortality: If expire_flag is 1 (patient died), mark as mortality
merged_df['MORTALITY'] = merged_df['EXPIRE_FLAG'] == 1

# If the death date ('dod') is within the ICU stay, count it as mortality
merged_df['MORTALITY'] = merged_df.apply(
    lambda row: 1 if row['EXPIRE_FLAG'] == 1 and pd.to_datetime(row['DOD']) >= row['INTIME'] and pd.to_datetime(row['DOD']) <= row['OUTTIME'] else row['MORTALITY'],
    axis=1
)

# Group by age and calculate the mean mortality rate for each age group
age_grouped = merged_df.groupby(pd.cut(merged_df['AGE_AT_ADDMISSION'], bins=range(0, 100, 10))).agg({'MORTALITY': 'mean'}).reset_index()

# Convert Interval objects to strings for better readability
age_grouped['AGE_AT_ADDMISSION'] = age_grouped['AGE_AT_ADDMISSION'].astype(str)

# Create a polar bar chart
fig = px.sunburst(age_grouped, path=['AGE_AT_ADDMISSION'], values='MORTALITY', title='Mortality Rate by Age Group')
fig.update_traces(textinfo='label+percent parent')
fig.update_layout(margin=dict(t=80, l=0, r=0, b=80))

# Show the figure
fig.show()


/var/folders/_c/ggv3f7q50ljf06zfzw29jmfh0000gn/T/ipykernel_4948/1848217840.py:37: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [321]:

# Connect to the SQLite database
conn = sqlite3.connect('../Assignment_MIMIC_SQL/database.db')

# Query to get ICU stays and discharge location information
query = """
SELECT icu.subject_id, icu.intime, icu.outtime, adm.discharge_location
FROM icustays icu
JOIN admissions adm ON icu.hadm_id = adm.hadm_id;
"""

# Load the query result into a pandas DataFrame
merged_df = pd.read_sql(query, conn)

# Close the connection after the query
conn.close()

# Convert 'intime' and 'outtime' to datetime
merged_df['INTIME'] = pd.to_datetime(merged_df['INTIME'])
merged_df['OUTTIME'] = pd.to_datetime(merged_df['OUTTIME'])

# Calculate ICU stay duration (LOS) in days
merged_df['LOS'] = (merged_df['OUTTIME'] - merged_df['INTIME']).dt.total_seconds() / (60 * 60 * 24)  # Convert to days

# Check for missing discharge location and drop those rows
merged_df = merged_df.dropna(subset=['DISCHARGE_LOCATION'])

# Create a box plot to visualize ICU LOS by discharge location
fig = px.box(merged_df, 
             x='DISCHARGE_LOCATION', 
             y='LOS', 
             title="ICU Length of Stay vs. Discharge Location", 
             labels={'LOS': 'ICU Length of Stay (Days)', 'DISCHARGE_LOCATION': 'Discharge Location'},
             color='DISCHARGE_LOCATION',
             boxmode='overlay')  # Overlay boxplots for better comparison

# Show the figure
fig.show()

In [322]:

# Fetch the data
conn = sqlite3.connect('../Assignment_MIMIC_SQL/database.db')

query = """
SELECT CE.SUBJECT_ID, CE.HADM_ID, CE.ITEMID, CE.VALUE, CE.CHARTTIME
FROM CHARTEVENTS CE
WHERE CE.ITEMID IN (220045, 220181) AND CE.SUBJECT_ID = 36
ORDER BY CE.CHARTTIME;
"""

chartevents_df = pd.read_sql(query, conn)

# Close the connection after the query
conn.close()


# Convert CHARTTIME to datetime
chartevents_df['CHARTTIME'] = pd.to_datetime(chartevents_df['CHARTTIME'])

# Map ITEMID to readable names (e.g., Heart Rate and SpO2)
chartevents_df['PARAMETER'] = chartevents_df['ITEMID'].map({220045: 'Heart Rate', 220181: 'Oxygen Saturation'})

# Pivot the data to separate Heart Rate and SpO2 into different columns
pivot_df = chartevents_df.pivot_table(index='CHARTTIME', 
                                      columns='PARAMETER', 
                                      values='VALUE', 
                                      aggfunc='first').reset_index()
                                      

# Create the time series graph
fig = px.line(pivot_df, x='CHARTTIME', y=['Heart Rate', 'Oxygen Saturation'],
              title="Heart Rate and Oxygen Saturation Over Time (Patient ID: 36)",
              labels={'CHARTTIME': 'Time', 'VALUE': 'Measurement Value'})
fig.show()



In [323]:

query = """
SELECT CE.SUBJECT_ID, CE.HADM_ID, CE.ITEMID, CE.VALUE, CE.CHARTTIME
FROM CHARTEVENTS CE
WHERE CE.ITEMID IN (220050, 220051) AND CE.SUBJECT_ID = 71582
ORDER BY CE.CHARTTIME;
"""

# Fetch the data
conn = sqlite3.connect('../Assignment_MIMIC_SQL/database.db')

# Fetch the data
chartevents_df = pd.read_sql(query, conn)

# Close the connection after the query
conn.close()

# Convert CHARTTIME to datetime
chartevents_df['CHARTTIME'] = pd.to_datetime(chartevents_df['CHARTTIME'])

# Map ITEMID to readable names (e.g., Systolic BP and Diastolic BP)
chartevents_df['PARAMETER'] = chartevents_df['ITEMID'].map({220050: 'Systolic BP', 220051: 'Diastolic BP'})

# Pivot the data to separate Systolic and Diastolic BP into different columns
pivot_df = chartevents_df.pivot_table(index='CHARTTIME', 
                                      columns='PARAMETER', 
                                      values='VALUE', 
                                      aggfunc='first').reset_index()

# Create the time series graph
fig = px.line(pivot_df, x='CHARTTIME', y=['Systolic BP', 'Diastolic BP'],
              title="Systolic and Diastolic Blood Pressure Over Time (Patient ID: 71582)",
              labels={'CHARTTIME': 'Time', 'VALUE': 'Blood Pressure (mmHg)'})
fig.show()

In [324]:

# Connect to the SQLite database
conn = sqlite3.connect('../Assignment_MIMIC_SQL/database.db')

# Query to get ICU length of stay and mortality flag data
query = """
SELECT icu.SUBJECT_ID, icu.HADM_ID, icu.LOS, p.EXPIRE_FLAG
FROM ICUSTAYS icu
JOIN PATIENTS p ON icu.SUBJECT_ID = p.SUBJECT_ID
"""

# Load the data into a pandas DataFrame
data = pd.read_sql(query, conn)
conn.close()

# Create a violin plot to visualize ICU Length of Stay by Mortality Status
fig = px.violin(data, 
                x='EXPIRE_FLAG', 
                y='LOS', 
                title="Distribution of ICU Length of Stay by Mortality",
                labels={'LOS': 'Length of Stay (Days)', 'EXPIRE_FLAG': 'Mortality (0 = Survived, 1 = Deceased)'},
                color='EXPIRE_FLAG',
                category_orders={'EXPIRE_FLAG': [0, 1]},
                box=True,  # Show box plot inside the violin
                points='all',  # Show individual data points
)

fig.update_layout(
    xaxis_title="Mortality Status (0 = Survived, 1 = Deceased)",
    yaxis_title="Length of Stay (Days)",
    height=500
)

fig.show()
